# to plate CSY

looking at functiosn to transform from CMM CSY to bar CSY using `matrix_from_3_points`

In [2]:
import numpy as np

import cmm_error_map.config.config as cf
import cmm_error_map.data_cmpts as dc


In [59]:
# very short bar for testing
bar = dc.ArtefactType(title="short bar", nballs=(4, 1), ball_spacing=100.0)

In [70]:
transform_mat = dc.matrix_from_vectors([50.0, 50.0, 50.0], [0.0, 0.0, 45.0])
transform_mat

array([[ 0.70710678, -0.70710678,  0.        , 50.        ],
       [ 0.70710678,  0.70710678,  0.        , 50.        ],
       [ 0.        ,  0.        ,  1.        , 50.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [71]:
xt, yt, zt = 0.0, 0.0, -250
prb = dc.Probe(title="test", name="test", length=np.array([xt, yt, zt]))
mmt = dc.Measurement(
    title="test",
    name="mmt_02",
    artefact=bar,
    transform_mat=transform_mat,
    probe=prb,
    cmm_nominal=None,
    cmm_dev=None,
    mmt_nominal=None,
    mmt_dev=None,
)

In [72]:
model = dc.model_parameters_dict.copy()
mmt.recalculate(model, dc.pmm_866.cmm_model)

In [73]:
mmt.mmt_nominal

array([[  0., 100., 200., 300.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.]])

In [74]:
mmt.mmt_nominal + mmt.mmt_dev

array([[ 0.00000000e+00,  1.00000000e+02,  2.00000000e+02,
         3.00000000e+02],
       [ 2.66453526e-15,  7.18521652e-15,  1.88113251e-14,
        -1.21951304e-14],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])

In [24]:
cmm_deform = mmt.cmm_nominal + mmt.cmm_dev
# assume bar got where it is by a rotation about its y axis
# a rotation about machine Z and an xyz shift.
# that is no rotation about bar axis.

# can take point  at (0, 100, 0) in bar csy and transform it into cmm csy using mmt.transform_mat?

# the rotation about y-axis is the angle between the bar axis and the x-y plane
# if this is 90 degrees the next bit doesn't work.
# the rotation about cmm z is the angle of the projection of the bar into the x-y plane
# but we know these roughly from mmt.transform_mat


xyz0 = cmm_deform[:, 0]
xyz1 = cmm_deform[:, -1]
bar_vx = xyz1 - xyz0
cmm_z = np.array([0.0, 0.0, 1.0])


In [18]:
np.dot(bar_vx, cmm_z)

0.0

In [27]:
cmm_deform = mmt.cmm_nominal + mmt.cmm_dev
xyz0 = cmm_deform[:, 0]
xyz1 = cmm_deform[:, -1]

# for third point
# take  (0, 100, 0) in bar csy and
# transform it into cmm csy using mmt.transform_mat
bar_c2 = np.array([0, 100, 0, 1])
xyz2 = mmt.transform_mat @ bar_c2


In [28]:
xyz2.shape

(4,)

In [33]:
xyz2 = xyz2[:3]
corners = np.vstack((xyz0, xyz1, xyz2))
corner_inds = [0, 1, 2]
mat = dc.matrix_from_3_points(corners, corner_inds)
cmm_deform1 = np.vstack((cmm_deform, np.ones((1, cmm_deform.shape[1]))))
mmt_deform = mat @ cmm_deform1

In [34]:
corners

array([[ 50.        ,  50.        ,  50.        ],
       [474.26406871, 474.26406871,  50.        ],
       [-20.71067812, 120.71067812,  50.        ]])

In [35]:
mat

array([[ 0.00000000e+00,  4.01943669e-16,  1.00000000e+00,
         2.07106781e+01],
       [-0.00000000e+00, -1.00000000e+00,  4.01943669e-16,
         4.74264069e+02],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -5.00000000e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [41]:
n1 = np.arange(21).reshape(3, -1)
n1

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20]])

In [42]:
n1[:, 0]

array([ 0,  7, 14])

In [44]:
np.vstack((n1[:, 0], n1[:, 1]))

array([[ 0,  7, 14],
       [ 1,  8, 15]])

In [45]:
np.hstack((n1[:, 0], n1[:, 1]))

array([ 0,  7, 14,  1,  8, 15])

In [46]:
np.stack((n1[:, 0], n1[:, 1]))

array([[ 0,  7, 14],
       [ 1,  8, 15]])

In [47]:
np.vstack((n1[:, 0], n1[:, 1])).T

array([[ 0,  1],
       [ 7,  8],
       [14, 15]])

In [ ]:
np.polynomial.Polynomial([1, 2, 3])(10)

321.0